In [1]:
import sys
sys.path.append("../")
import nanonis
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt
import xlrd
import lmfit
from lmfit import Model
import matplotlib.colors as colors
from scipy import constants as const
%matplotlib qt
spectra=nanonis.biasSpectroscopy()

# Useful functions

In [ ]:
def fermi(x,T):
    return np.divide(1,1+np.exp(x/T))


def BCS(x,delta,eta):
    return np.sign(x)*np.imag(np.divide(np.abs(x+1j*eta),np.sqrt(delta**2-(x+1j*eta)**2)))

def coulomb(x,Ec,Q0):
    return np.heaviside(x-Ec-Q0,0)+np.heaviside(-x-Ec+Q0,0)

def conv_dynes(x,delta1,delta2,eta,T,Ec,Q0):
    y=np.linspace(-5*(delta1+delta2),5*(delta1+delta2),5000)
    a,b=np.meshgrid(y,y)
    cou=np.gradient(-np.dot(BCS(a+b,delta1,eta)*fermi(a+b,T),coulomb(y,Ec,Q0))+np.dot(BCS(a+b,delta1,eta),fermi(y,T)*coulomb(y,Ec,Q0)))
    a,b=np.meshgrid(y,x)
    z=np.gradient(-np.dot(BCS(a+b,delta2,eta)*fermi(a+b,T),cou)+np.dot(BCS(a+b,delta2,eta),cou*fermi(y,T)))
    return z/z[0]

def temp(x,delta1,delta2,eta,T):
    y=np.linspace(-5*(delta1+delta2),5*(delta1+delta2),5000)
    a,b=np.meshgrid(y,x)
    return -np.dot(BCS(a+b,delta2,eta)*fermi(a+b,T),BCS(y,delta1,eta))+np.dot(BCS(a+b,delta2,eta),BCS(y,delta1,eta)*fermi(y,T))

def temp2(V,delta1,delta2,eta,T):
    y=np.linspace(-5*(delta1+delta2),5*(delta1+delta2),5000)
    return np.sum(-BCS(y+V,delta2,eta)*fermi(y+V,T)*BCS(y,delta1,eta)+BCS(y+V,delta2,eta)*BCS(y,delta1,eta)*fermi(y,T))

def gap(x,delta1,Bc1,Ec,a):
    d1=(0.0013*np.real(np.sqrt(1-((x+0.0000*1j)/800)**2)))*np.heaviside(-x+800,0)
    d2=delta1*np.real(np.sqrt(1-((x+0.0000*1j)/Bc1)**2))*np.heaviside(-x+Bc1,0)
    d3=(-a*x)*np.heaviside(x-Bc1,0)
    return Ec+d1+d2+d3
   
def coulomb_T(x,Ec,T):
    y=np.linspace(-3*np.min(x),3*np.min(x),2000)
    a,b=np.meshgrid(y,x)
    return np.gradient(-np.dot(fermi(a+b,T),coulomb(y,Ec,Q0))+fermi(y,T)*coulomb(y,Ec,Q0))

def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return idx

def delta_B(B,Delta_0,Bc):
    if B<Bc:
        return Delta_0*np.sqrt(1-(B/Bc)**2)
    else:
        return 0


# File management #

In [379]:
data=pd.read_excel("C:/Users/jonor/Documents/GitHub/nanonis/Order.xls")

In [380]:
Bfield_files=data.iloc[:,2]
Bfields=data.iloc[:,3]
size_dat=data.iloc[:,7]
height_dat=data.iloc[:,8]
temp=data.iloc[:,6]
temp_files=data.iloc[:,5]
josephson=data.iloc[:,4]
path=data.iloc[:,10]

# B Field 

In [ ]:
## All files
dat=[1,2,3,4,5,6,7,8,10,11,12,13,14,15,16,17,18,19,21,23,24]
BFields=[]
BFiles=[]
size=[]
height=[]
for i in dat:
    a=[int(j) for j in Bfields[i].split(',')]
    b=Bfield_files[i].split(',')    
    files=[path+i for i in b]
    BFields.append(a)
    BFiles.append(files)
    size.append(size_dat[i])
    height.append(height_dat[i])
size=np.array(size)
height=np.array(height)

In [ ]:
i=12
a=[int(j) for j in Bfields[i].split(',')]
b=Bfield_files[i].split(',')
files=[path+i for i in b]


In [ ]:
w_B=[0,1,2,3,4,5,6,7,8,10,11,12,13,14,15,16,17,18,19,21,23,24]
Bfiles=[]
size=[]
B=[]

for j in w_B:
    a=[int(i) for i in Bfields[j].split(',')]
    b=Bfield_files[j].split(',')
    files=[path+i for i in b]
    B.append(a)
    #field0=a.index(0)
    #files_B0.append(files[field0])
    Bfiles.append(files)
    size.append(size_dat[j])


### Plots

In [ ]:
y=[]
for i in files:
    spectra.load(i)
    y.append(spectra.conductance/spectra.conductance[0])
x=np.array(spectra.bias)
plt.imshow(y,aspect='auto',origin='lower',extent=(1000*x[0],1000*x[-1],a[-1],a[0]))
plt.ylabel('B(mT)')
plt.xlabel('Bias (meV)')

### Some info

We look at the B field point after 800 mT to see the effect of B field on the coulomb gap

In [ ]:
def coulomb_T(x,Ec,Q0,T):
    return fermi(Ec-x+Q0,T)+fermi(Ec+x-Q0,T)
def coulomb(x,Ec,Q0):
    T=0.0003
    return fermi(Ec-x+Q0,T)+fermi(Ec+x-Q0,T)
def coulomb_T2(x,Ec,Q0,T):
    return np.sinh(np.abs(x)/T)/(np.cosh(x/T)+np.cosh(Ec/T))

h=0
Bfield=[800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300,2400,2500]
Gap=[]
Asym=[]
Si=[]
for k in Bfield:
    Ec=[]
    Q0=[]
    size2=[]
    for i in range(len(Bfiles)):
        for j in range(len(Bfiles[i])):
            if np.abs(B[i][j]-k)<=0.00001:
                size2.append(size[i])
                spectra.load(Bfiles[i][j])
                x=np.array(spectra.bias)
                y=np.array(spectra.conductance)
                y=y/y[0]
                #values=sc.optimize.curve_fit(coulomb_T,x,y,p0=[0.001,0.00,0.0003])[0]
                values=sc.optimize.curve_fit(coulomb,x,y,p0=[0.001,0.00])[0]
                #yy=coulomb_T(x,values[0],values[1],values[2])
                #yy=coulomb(x,values[0],values[1])
                Ec.append(values[0])
                Q0.append(values[1])
                #ther.append(values[2])
                #plt.plot(x,yy+h)
                #plt.plot(x,y+h)
                #h+=1
            else:
                continue
    Gap.append(Ec)
    Asym.append(Q0)
    Si.append(size2)

In [ ]:
def coulomb_T(x,Ec,Q0,T):
    return fermi(Ec-x+Q0,T)+fermi(Ec+x-Q0,T)
def coulomb(x,Ec,Q0):
    T=0.0003
    return fermi(Ec-x+Q0,T)+fermi(Ec+x-Q0,T)
def coulomb_T2(x,Ec,Q0,T):
    return np.sinh(np.abs(x)/T)/(np.cosh(x/T)+np.cosh(Ec/T))

h=0
Bfield=[800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300,2400,2500]
Gap=[]
Asym=[]
Si=[]
for k in Bfield:
    Ec=[]
    Q0=[]
    size2=[]
    for i in range(len(Bfiles)):
        for j in range(len(Bfiles[i])):
            if np.abs(B[i][j]-k)<=0.00001:
                size2.append(size[i])
                spectra.load(Bfiles[i][j])
                x=np.array(spectra.bias)
                y=np.array(spectra.current)
                y=y/y[0]
                xx=[]
                for p in range(len(y)):
                    if np.abs(y[p]/y[0])<0.03:
                        xx.append(x[p])
                Ec.append(np.array(xx)[0]-np.array(xx)[-1])
            else:
                continue
        
    Gap.append(Ec)
    Asym.append(Q0)
    Si.append(size2)

In [ ]:

plt.scatter(Si[4],Gap[4])


In [ ]:
def coulomb(x,Ec,Q0):
    T=0.0003
    return fermi(Ec-x+Q0,T)+fermi(Ec+x-Q0,T)


emp=[]
for i in Bfiles:
    en=[]
    for j in i:
        en.append(0.0)
    emp.append(en)
Ec=emp
Q0=emp
for i in range(len(Bfiles)):
    for j in range(len(Bfiles[i])):
        spectra.load(Bfiles[i][j])
        x=np.array(spectra.bias)
        y=np.array(spectra.conductance)
        y=y/y[0]
        values=sc.optimize.curve_fit(coulomb,x,y,p0=[0.001,0.00])[0]
        #Ec[i][j]=values[0]
        Q0[i][j]=values[1]
        Ec[i][j]=values[0]
        
        


In [ ]:
emp=[]
for i in Bfiles:
    en=[]
    for j in i:
        en.append(0.0)
    emp.append(en)
Ec=emp
Q0=emp
for i in range(len(Bfiles)):
    for j in range(len(Bfiles[i])):
        spectra.load(Bfiles[i][j])
        x=np.array(spectra.bias)
        y=np.array(spectra.current)
        y=y/y[0]
        xx=[]
        for p in range(len(y)):
            if np.abs(y[p]/y[0])<0.01:
                xx.append(x[p])
        Ec[i][j]=np.array(xx)[0]-np.array(xx)[-1]        

In [ ]:
i=4
plt.plot(xx[i],yy[i])

In [ ]:
xx=[]
yy=[]
for j in range(22):
    x=[]
    y=[]
    si=size[j]
    for i in range(len(B[j])):
        if B[j][i]>=1000:
            x.append(B[j][i])
            y.append(Ec[j][i])
        else:
            continue
    xx.append(x)
    yy.append(y)
    a,b=np.polyfit(x,y,1)
    plt.plot(si,a,'o')

In [ ]:

plt.plot(xx[4],yy[4])

In [ ]:
plt.rcParams["axes.prop_cycle"] = plt.cycler("color", plt.cm.viridis(np.linspace(0,1,10)))
#for i in range(18):
#    plt.scatter(Si[:][i],1000*np.array(Asym[:][i]))
plt.scatter(Si[0],1000*np.array(Asym[0]))
plt.scatter(Si[17],1000*np.array(Asym[17]))


#plt.scatter(size2,1000*np.array(Q08),c='#ff7f0e',label='T=8K')
#plt.xlabel('Island Area ($nm^2$)')
#plt.ylabel('$Q_0$ ($meV$)')
#plt.legend()

In [ ]:
np.sum(np.abs(0-np.array(Asym[0])))/len(np.array(Asym[0])),np.sum(np.abs(0-np.array(Asym[17])))/len(np.array(Asym[17]))

### Some info

The simplest way to get the gao is to calculate it from the current. We take the points for which the current is zero.



In [ ]:
h=0
Ec=[]
Eg=[]
for k in range(len(BFiles)):
    cur=[]
    for i in BFiles[k]:
        spectra.load(i)
        cur.append(spectra.current)

    t=[]
    for j in range(len(BFiles[k])):
        y=np.array(cur[j])
        x=np.array(spectra.bias)
        xx=[]
        for i in range(len(y)):
            if np.abs(y[i]/y[0])<0.04:
                xx.append(x[i])
        t.append(xx[0]-xx[-1])
    T800=BFields[k].index(800)
    T0=BFields[k].index(0)
    Ec.append(t[T0])


plt.plot(size,Ec,'bo')


### Solme info

The critical field for the tip is around B=800mT, while the critical field for the island should be around B=350mT. Althought the first statement is prety robust, the second is not. 

The following program fit the spectra with a simple convolution. The free parameters will be the coulomb gap (Ec) the shift (Q0) and the dynes parameter (eta). We will suppose that the gaps decrease following the BCS theory with the critical fields that we wrote in the previous line. This first program is not a realistic fir of the system, we will only use it to get an idea of the volution of the full gap of the measurement, i.e., E+Delta1+Delta2. We will not make assupmtions about any of this parameters from the results of this analysis.

In [ ]:

def delta(Delta,B,Bc):
    return Delta*np.sqrt(1-(B/Bc)**2)

def conv_dynes(x,delta,eta,Ec):
    T=delta/10
    Q0=-0.00025
    y=np.linspace(-10*delta,10*delta,2000)
    a,b=np.meshgrid(y,x)
    z=np.gradient(-np.dot(BCS(a+b,delta,eta)*fermi(a+b,T),coulomb(y,Ec,Q0))+np.dot(BCS(a+b,delta,eta),fermi(y,T)*coulomb(y,Ec,Q0)))
    return z/z[0]


cond=[]
for i in files:
    spectra.load(i)
    cond.append(spectra.conductance)

gap=[]
for j in range(0,10):
    y=np.array(cond[j])[110:290]/np.array(cond[j])[110]
    x=np.array(spectra.bias)[110:290]
    temp=fit(conv_dynes,x,y,p0=[0.0013,0.0001,0.0012])
    plt.plot(x,y+0.5*j)
    p=temp[0]
    z=conv_dynes(x,p[0],p[1],p[2])
    plt.plot(x,z+0.5*j)
    gap.append(p[0]+p[2])
    print(j)

# Temperature dependence

In [ ]:
data=[8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]
T=[]
files=[]
size=[]
height=[]

for j in data:
    a=temp[j].split(',')
    T.append([float(i) for i in a])
    b=temp_files[j].split(',')
    files.append([path[j]+i for i in b])
    size.append(size_dat[j])
    height.append(height_dat[j])

In [ ]:
f=files[0][0]
spectra.load(f)

x=np.array(spectra.bias)
y=np.array(spectra.conductance)
y=y/np.sum(y)
plt.plot(x,y)


In [ ]:
Ec=[]
Q0=[]
ther=[]
h=0
temperature=8.0
size2=[]
for i in range(len(files)):
    for j in range(len(files[i])):
        if np.abs(T[i][j]-temperature)<=0.00001:
            size2.append(size[i])
            spectra.load(files[i][j])
            x=np.array(spectra.bias)
            y=np.array(spectra.conductance)
            y=y/y[0]
            #values=sc.optimize.curve_fit(coulomb_T,x,y,p0=[0.001,0.00,0.002])[0]
            values=sc.optimize.curve_fit(coulomb,x,y,p0=[0.001,0.00])[0]
            #yy=coulomb_T(x,values[0],values[1],values[2])
            yy=coulomb(x,values[0],values[1])
            Ec.append(values[0])
            Q0.append(values[1])
            #ther.append(values[2])
            #plt.plot(x,yy+h)
            #plt.plot(x,y+h)
            h+=1
        else:
            continue
        


plt.scatter(size2,1000*np.array(Ec),c='#ff7f0e',label='T=7K')

In [ ]:
np.mean(ther)

In [ ]:
Ec7
size7
size8
Ec8
Q08
Q07

In [ ]:
plt.scatter(size7,1000*np.array(Q07),c='#1f77b4',label='T=7K')
x=np.linspace(100,2000,1000)
#plt.plot(x,1/(0.001*x))
plt.scatter(size8,1000*np.array(Q08),c='#ff7f0e',label='T=8K')
plt.xlabel('Island Area ($nm^2$)')
plt.ylabel('$E_C$ ($meV$)')
plt.legend()
#plt.scatter(np.array(size),1000*np.array(Q0),c='#ff7f0e')
#plt.scatter(np.array(size),1000*np.array(ther),c='#2ca02c')

In [ ]:
np.sum(np.array(ther))/len(ther)

# Josephson 

In [ ]:
Jos_files=[]
JSize=[]
a=[0,1,2,3,4,5,6,7,8,11,12,13,14,15,16,17,18,19,21,22,23,24,25,26,27,28,29,30,31,32,33]
for j in a:
    Jos_files.append(np.array([path[j]+i for i in josephson[j].split(',')]))
    JSize.append(size_dat[j])
#JSize, Jos_files = (list(t) for t in zip(*sorted(zip(JSize,Jos_files))))

In [ ]:
fig, axs = plt.subplots(5,7)
h=0
j=0
for i in range(len(Jos_files)):
    spectra.load(Jos_files[i][-1])
    x=np.array(spectra.bias)
    y=np.array(spectra.current)
    #yt=np.sum(y)
    #y=y/yt
    a1=min(find_nearest(x,-0.0015),find_nearest(x,0.0015))
    a2=max(find_nearest(x,-0.0015),find_nearest(x,0.0015))
    axs[j,h].plot(1000*x[a1:a2],y[a1:a2]/np.abs(y[a1]),label=JSize[i])
    y=np.array(spectra.conductance)
    a1=min(find_nearest(x,-0.0015),find_nearest(x,0.0015))
    a2=max(find_nearest(x,-0.0015),find_nearest(x,0.0015))
    axs[j,h].plot(1000*x[a1:a2],y[a1:a2]/np.abs(y[a1]),label=JSize[i])

    #axs[j,h].plot(1000*x,y,label=JSize[i])
    #axs[j,h].plot(1000*x,y,label=JSize[i])
    h+=1
    if h==7:
        h=0
        j+=1


In [ ]:
ax1 = plt.subplot2grid((2,8), (0,0), colspan=2)
ax2 = plt.subplot2grid((2,8), (0,3), colspan=2)
ax3 = plt.subplot2grid((2,8), (0,6), colspan=2)
ax4 = plt.subplot2grid((2,8), (1,2), colspan=2)
ax5 = plt.subplot2grid((2,8), (1,5), colspan=2)
ax=[ax1,ax2,ax3,ax4,ax5]
h=0
for i in  [0,1,3,6,18]:
    spectra.load(Jos_files[i][-1])
    x=np.array(spectra.bias)
    y=np.array(spectra.current)
    #x0=float(spectra.header.get('Lock-in>Amplitude'))
    #print(x0)
    #yt=np.sum(y)
    #y=y/yt
    #ax[h].plot(1000*x,y/10e-9,label=JSize[i],linewidth=3)
    ax[h].set_xlabel('Bias (meV)')
    ax[h].set_ylabel('Current (nA)')
    a1=min(find_nearest(x,-0.0005),find_nearest(x,0.0005))
    a2=max(find_nearest(x,-0.0005),find_nearest(x,0.0005))
    x=x[a1:a2]
    y=y[a1:a2]
    ax[h].plot(x,y/10e-9,label=JSize[i],linewidth=3)
    ax[h].vlines(0.0001,np.min(y)/10e-9,np.max(y)/10e-9)
    ax[h].vlines(-0.0001,np.min(y)/10e-9,np.max(y)/10e-9)
    #ax[h].vlines(0.0,np.min(y)/10e-9,np.max(y)/10e-9)
    ax[h].axhline(0.0)
    #axs[j,h].plot(1000*x,y,label=JSize[i])
    h+=1

In [ ]:
sure=[1,2,4,6,18]
meh=[0,8]

In [ ]:
spectra.load(Jos_files[30][-1])
x=np.array(spectra.bias)
y=np.array(spectra.current)
#x0=float(spectra.header.get('Lock-in>Amplitude'))
a1=min(find_nearest(x,-0.0005),find_nearest(x,0.0005))
a2=max(find_nearest(x,-0.0005),find_nearest(x,0.0005))
y=y[a1:a2]
x=x[a1:a2]
plt.plot(x,y)
#plt.vlines(0,np.min(y),np.max(y))
plt.vlines(x0,np.min(y),np.max(y))
plt.vlines(-x0,np.min(y),np.max(y))
plt.axhline(0.0)


In [ ]:
spectra.load(Jos_files[4][0])
x=np.array(spectra.bias)
y=np.array(spectra.conductance)
y=y/y[0]
plt.plot(1000*x,y)
R1=10000
R2=1
C1=0.01
C2=0.6
Q0=0.0
Delta=0.5
delta_t=1.3
delta_s=0.1
T=0.1
eta=0.1

y=currentD(1000*x,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta)
y=y/y[0]
plt.plot(1000*x,y)


In [ ]:
Jos_files[0][-1]

In [ ]:
current0=[]
resistance=[]
a=range(len(Jos_files))
for i in a:
    spectra.load(Jos_files[i][-1])
    x=np.array(spectra.bias)
    y=np.array(spectra.current)
    #yt=np.sum(y)
    #y=y/yt
    a1=find_nearest(x,0.0)
    current0.append(y[a1])
    resistance.append(np.mean(x[len(x)-10:len(x)]/y[len(x)-10:len(x)]))
    #axs[j,h].plot(1000*x,y,label=JSize[i])

plt.scatter(a,np.array(current0)/10e-12)




## Good Josephson

In [ ]:
Good_Jos_files=[]
Good_JSize=[]
a=[0,1,2,3,4,6,8,11,12,13,14,15,16,21,22,23,24,25,26,27,28,29,32]
for j in a:
    Good_Jos_files.append(np.array([path[j]+i for i in josephson[j].split(',')]))
    Good_JSize.append(size_dat[j])
Good_JSize, Good_Jos_files = (list(t) for t in zip(*sorted(zip(Good_JSize,Good_Jos_files))))

In [ ]:
fig, axs = plt.subplots(4,6)
h=0
j=0
for i in range(len(Good_Jos_files)):
    spectra.load(Good_Jos_files[i][-1])
    x=np.array(spectra.bias)
    y=np.array(spectra.current)
    #yt=np.sum(y)
    #y=y/yt
    a1=min(find_nearest(x,-0.0015),find_nearest(x,0.0015))
    a2=max(find_nearest(x,-0.0015),find_nearest(x,0.0015))
    axs[j,h].plot(1000*x[a1:a2],y[a1:a2]/np.abs(y[a1]),label=JSize[i])
    y=np.array(spectra.conductance)
    a1=min(find_nearest(x,-0.0015),find_nearest(x,0.0015))
    a2=max(find_nearest(x,-0.0015),find_nearest(x,0.0015))
    axs[j,h].plot(1000*x[a1:a2],y[a1:a2]/np.abs(y[a1]),label=JSize[i])

    #axs[j,h].plot(1000*x,y,label=JSize[i])
    #axs[j,h].plot(1000*x,y,label=JSize[i])
    h+=1
    if h==6:
        h=0
        j+=1


In [ ]:
Sym_Jos_files=[]
Sym_JSize=[]
a=[1,3,4,13,15,16,21,22,25,26,32]
for j in a:
    Sym_Jos_files.append(np.array([path[j]+i for i in josephson[j].split(',')]))
    Sym_JSize.append(size_dat[j])
#Sym_JSize, Sym_Jos_files = (list(t) for t in zip(*sorted(zip(Sym_JSize,Sym_Jos_files))))

In [ ]:
ax1 = plt.subplot2grid((4,6), (0,0), colspan=2)
ax2 = plt.subplot2grid((4,6), (0,2), colspan=2)
ax3 = plt.subplot2grid((4,6), (0,4), colspan=2)
ax4 = plt.subplot2grid((4,6), (1,0), colspan=2)
ax5 = plt.subplot2grid((4,6), (1,2), colspan=2)
ax6 = plt.subplot2grid((4,6), (1,4), colspan=2)
ax7 = plt.subplot2grid((4,6), (2,0), colspan=2)
ax8 = plt.subplot2grid((4,6), (2,2), colspan=2)
ax9 = plt.subplot2grid((4,6), (2,4), colspan=2)
ax10 = plt.subplot2grid((4,6), (3,1), colspan=2)
ax11 = plt.subplot2grid((4,6), (3,3), colspan=2)
ax=[ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8,ax9,ax10,ax11]
for i in range(len(Sym_Jos_files)):
    spectra.load(Sym_Jos_files[i][-1])
    x=np.array(spectra.bias)
    y=np.array(spectra.current)
    #yt=np.sum(y)
    #y=y/yt
    a1=min(find_nearest(x,-0.002),find_nearest(x,0.002))
    a2=max(find_nearest(x,-0.002),find_nearest(x,0.002))
    R=int(np.mean(x[len(x)-10:len(x)]/y[len(y)-10:len(y)]))
    ax[i].plot(1000*x[a1:a2],y[a1:a2]/np.abs(y[a1]),label=R)
    y=np.array(spectra.conductance)
    a1=min(find_nearest(x,-0.002),find_nearest(x,0.002))
    a2=max(find_nearest(x,-0.002),find_nearest(x,0.002))
    ax[i].plot(1000*x[a1:a2],y[a1:a2]/np.abs(y[a1]),label=JSize[i])
    ax[i].legend()
    


In [ ]:
ax1 = plt.subplot2grid((4,6), (0,0), colspan=2)
ax2 = plt.subplot2grid((4,6), (0,2), colspan=2)
ax3 = plt.subplot2grid((4,6), (0,4), colspan=2)
ax4 = plt.subplot2grid((4,6), (1,0), colspan=2)
ax5 = plt.subplot2grid((4,6), (1,2), colspan=2)
ax6 = plt.subplot2grid((4,6), (1,4), colspan=2)
ax7 = plt.subplot2grid((4,6), (2,0), colspan=2)
ax8 = plt.subplot2grid((4,6), (2,2), colspan=2)
ax9 = plt.subplot2grid((4,6), (2,4), colspan=2)
ax10 = plt.subplot2grid((4,6), (3,1), colspan=2)
ax11 = plt.subplot2grid((4,6), (3,3), colspan=2)
ax=[ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8,ax9,ax10,ax11]
for i in range(len(Sym_Jos_files)):
    spectra.load(Sym_Jos_files[i][0])
    x=np.array(spectra.bias)
    y=np.array(spectra.current)
    #yt=np.sum(y)
    #y=y/yt
    R=int(np.mean(x[len(x)-20:len(x)]/y[len(y)-20:len(y)]))
    ax[i].plot(1000*x,y/np.abs(y[0]),label=R)
    y=np.array(spectra.conductance)
    a1=min(find_nearest(x,-0.002),find_nearest(x,0.002))
    a2=max(find_nearest(x,-0.002),find_nearest(x,0.002))
    ax[i].plot(1000*x,y/np.abs(y[0]),label=JSize[i])
    ax[i].legend()

In [ ]:
spectra.load(Sym_Jos_files[3][0])
x=np.array(spectra.bias)
y=np.array(spectra.conductance)

plt.plot(1000*x,y/np.sum(np.abs(y)))
R1=10000
R2=1
C1=0.01
C2=0.9
Delta=1.25
Q0=0.0
T=0.1
z=currentD(1000*x,R1,R2,C1,C2,Q0,Delta,T)
plt.plot(1000*x,z/np.sum(np.abs(z)))

# Islans by island

In [381]:
dat=[ 8, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 23, 24]
T=[]
Tfiles=[]
Bfiles=[]
Jfiles=[]
size=[]
height=[]
Bfiles=[]
B=[]

for j in dat:

    T.append([float(i) for i in temp[j].split(',')])

    Tfiles.append([path[j]+i for i in temp_files[j].split(',')])

    size.append(size_dat[j])

    height.append(height_dat[j])

    B.append([int(i) for i in Bfields[j].split(',')])

    Bfiles.append([path[j]+i for i in Bfield_files[j].split(',')])

    Jfiles.append([path[j]+i for i in josephson[j].split(',')])




### T dependence

In [ ]:
i=12
h=0
for j in range(len(Tfiles[i])):
    spectra.load(Tfiles[i][j])
    x=np.array(spectra.bias)
    y=np.array(spectra.conductance)
    y=y/y[0]
    plt.plot(1000*x,y+1.4-0.2*h,linewidth=2,label=T[i][j])
    plt.xlim(-10,10)
    plt.xlabel('Bias (meV)')
    plt.ylabel('DoS (a.u.)')
    plt.legend(loc=1)
    h+=1

### B dependence

In [ ]:
i=12
h=0
for j in range(len(Bfiles[i])):
    spectra.load(Bfiles[i][j])
    x=np.array(spectra.bias)
    y=np.array(spectra.conductance)
    y=y/y[0]
    plt.plot(1000*x,y+0.2*h,linewidth=2)
    plt.xlim(-10,10)
    plt.xlabel('Bias (meV)')
    plt.ylabel('DoS (a.u.)')
    h+=1
plt.title('B='+str(B[i][0])+'T to '+str(B[i][-1])+ 'T')

In [ ]:

y=[]

for i in Bfiles[1]:
    spectra.load(i)
    y.append(spectra.conductance/spectra.conductance[0])
x=np.array(spectra.bias)
plt.imshow(y,aspect='auto',origin='lower',extent=(1000*x[0],1000*x[-1],a[-1],a[0]))
plt.ylabel('B(mT)')
plt.xlabel('Bias (meV)')

### B dependence Fit

We first take spectra for B>800mT and fit it with the model for a metal

In [392]:
ax = plt.axes(projection='3d')
for i in range(5,6):
    h=0
    C1=[]
    #C2=[]
    Q0=[]
    ther=[]
    B800=B[i].index(800)
    for j in Bfiles[i][B800:-1]:
        spectra.load(j)
        x=np.array(spectra.bias)[150:250]
        y=np.array(spectra.conductance)[150:250]
        y=y/np.sum(np.abs(y))
        #plt.plot(1000*x,y+0.001*h,linewidth=2)
        par=sc.optimize.curve_fit(lambda xx, C1,Q0: current(xx, 1,10000, C1,0.001,Q0,0.1), 1000*x, y,p0=[0.5,0.0],bounds=([0.01,-0.5],[2.0,0.5]))[0]
        #a=current(1000*x,1,10000,par[0],0.0001,par[1],par[2])
        #plt.plot(1000*x,a+0.001*h)
        h+=1
        C1.append(par[0])
        Q0.append(par[1])
        #C2.append(par[3])
        #plt.xlim(-10,10)
        #plt.xlabel('Bias (meV)')
        #plt.ylabel('DoS (a.u.)')
        h+=1
    #plt.plot(B[i][B800:-1],C1,label=str(size[i]))
    C1=np.array(C1)
    ax.plot3D(np.full(len(C1),size[i]),B[i][B800:-1] , 1/C1)
#ax1 = plt.subplot2grid((2,5), (0,0), colspan=2)
#ax2 = plt.subplot2grid((2,5), (0,3), colspan=2)
#ax3 = plt.subplot2grid((2,4), (1,1), colspan=2)
#ax1.plot(B[i][B800:-1],C1,label='$C_1$')
#ax1.legend()
#ax2.plot(B[i][B800:-1],ther,label='$T$')
#ax2.legend()
#ax3.plot(B[i][B800:-1],Q0,label='$Q_0$')
#ax3.legend()
plt.legend()

No handles with labels found to put in legend.


In [ ]:
i=1
h=0
C1=[]
#C2=[]
Q0=[]
ther=[]
cutt=np.arange(10,200,10)
B800=B[i].index(800)
for cut in cutt:
    C1=[]
    er=[]
    for j in Bfiles[i][B800:-1]:
        spectra.load(j)
        x=np.array(spectra.bias)[200-cut:200+cut]
        y=np.array(spectra.current)[200-cut:200+cut]
        y=y/np.abs(y[0])
        #plt.plot(1000*x,y+1*h,linewidth=2)
        par,error=sc.optimize.curve_fit(lambda xx, C1,Q0: current2(xx, 1,10000, C1,0.001,Q0,0.1), 1000*x, y,p0=[0.5,0.0],bounds=([0.01,-0.5],[15.0,0.5]))
        #a=current2(1000*x,1,10000,par[0],0.0001,par[1],0.1)
        #plt.plot(1000*x,a+1*h)
        h+=1
        C1.append(par[0])
        Q0.append(par[1])
        #ther.append(par[2])
        #C2.append(par[3])
        #plt.xlim(-10,10)
        #plt.xlabel('Bias (meV)')
        #plt.ylabel('DoS (a.u.)')
        print(np.sqrt(np.diag(error)))
        er.append(np.sqrt(np.diag(error)))
    plt.plot(1/np.array(C1))


In [ ]:
er


In [ ]:
spectra.load(Bfiles[i][B800:-1][10])
x=np.array(spectra.bias)
y=np.array(spectra.current)
y=y/y[0]
plt.plot(1000*x,y)
a=current2(1000*x,1,10000,20,0.0001,par[1],0.1)
a=current(1000*x,1,10000,20,0.0001,par[1],0.1)
plt.plot(1000*x,a)

We try a different method to get the gap

In [ ]:
def gap_size(x,y,epsilon):
    xx=[]
    for i in range(len(y)):
        if np.abs(y[i])<epsilon:
            xx.append(x[i])
    return np.abs(xx[0]-xx[-1])
Eg=[]
#for j in Bfiles[i][B800:-1]:
#    spectra.load(j)
#    x=np.array(spectra.bias)
#    y=np.array(spectra.current)
#    Eg.append(gap_size(1000*x,y/y[0],0.01))
   
for i in range(0,1):
    Eg=[]
    B800=B[i].index(800)
    for j in Bfiles[i][B800:-1]:
        spectra.load(j)
        x=np.array(spectra.bias)
        y=np.array(spectra.current)
        y=y/y[0]
        Eg.append(gap_size(1000*x,y,max(y)/50)) 
    Eg=np.array(Eg)
    plt.plot(B[i][B800:-1] , Eg,label=str(size[i]))
plt.legend()


In [ ]:

for k in [1,10,50,100,150,300]:
    Eg=[]
    B800=B[0].index(800)
    for j in Bfiles[i][B800:-1]:
        spectra.load(j)
        x=np.array(spectra.bias)
        y=np.array(spectra.current)
        y=y/y[0]
        Eg.append(gap_size(1000*x,y,max(y)/k)) 
    Eg=np.array(Eg)
    plt.plot(B[0][B800:-1] , Eg,label=str(k))
plt.legend()

We will look at the spectra starting from 200mT. Taking into account the superconducting tip

In [ ]:

i=11
h=0
Delta=[]
C2=[]
Q0=[]
ther=[]
B200=B[i].index(200)
B800=B[i].index(800)
for j in Bfiles[i][B200:B800]:
    print(j)
    spectra.load(j)
    x=np.array(spectra.bias)
    y=np.array(spectra.conductance)
    y=y/np.sum(np.abs(y))
    plt.plot(1000*x,y+0.001*h,linewidth=2)
    par=sc.optimize.curve_fit(lambda xx, Delta,Q0,T,C2: currentD(xx, 10000,1, 0.001,C2,Q0,Delta,0.0,T,0.1), 1000*x, y,p0=[1.0,0.0,0.1,0.5],bounds=([0.0,-0.45,0.1,0.1],[1.0,0.45,0.3,1.0]))[0]
    a=current(1000*x,1,10000,par[0],par[3],par[1],par[2])
    plt.plot(1000*x,a+0.001*h)
    h+=1
    Delta.append(par[0])
    Q0.append(par[1])
    ther.append(par[2])
    C2.append(par[3])
    plt.xlim(-10,10)
    plt.xlabel('Bias (meV)')
    plt.ylabel('DoS (a.u.)')
    h+=1

In [ ]:
ax1 = plt.subplot2grid((2,5), (0,0), colspan=2)
ax2 = plt.subplot2grid((2,5), (0,3), colspan=2)
ax3 = plt.subplot2grid((2,4), (1,1), colspan=2)
#ax4 = plt.subplot2grid((2,5), (1,3), colspan=2)
ax1.plot(B[i][B800:-1],C1,label='$C_1$')
ax1.legend()
ax2.plot(B[i][B800:-1],C2,label='$C_2$')
ax2.legend()
ax3.plot(B[i][B800:-1],ther,label='$T$')
ax3.legend()
ax4.plot(B[i][B800:-1],Q0,label='$Q_0$')
ax4.legend()

In [ ]:
spectra.load(Bfiles[1][0])
x=np.array(spectra.bias)
y=np.array(spectra.conductance)
y=y/y[0]
plt.plot(1000*x,y)
Q0=-1.1
V=np.linspace(1000*x[-1],1000*x[0],100)
z=currentD(V, 10000,1, 0.001,1.5,Q0,0.9,1.3,0.0,0.1,0.1)
z=z/z[0]
plt.plot(V,z)

### Josephson

In [ ]:
R1=1000000
R2=1
C1=0.001
C2=3
Q0=np.linspace(0,2,20)
Delta=0.6
delta_t=1.3
delta_s=0.0
T=0.1
eta=0.05
spectra.load(Jfiles[9][0])
x=np.array(spectra.bias)
y=np.array(spectra.conductance)
y=y/y[0]
#plt.plot(1000*x,y)
V=np.linspace(-2,2,30)
pp0=[]
pp1=[]
pp_1=[]
pp2=[]
pp_2=[]
for i in Q0:
    print(i)
    p0,pn,p_n=all_pD(V,R1,R2,C1,C2,i,Delta,delta_t,T,eta,3)
    pp0.append(p0[15])
    pp1.append(pn[0][0])
    pp2.append(pn[1][0])
    pp_1.append(p_n[0][15])
    pp_2.append(p_n[1][15])

plt.scatter(Q0,pp0,label='0')
plt.scatter(Q0,pp1,label='1')
plt.scatter(Q0,pp_1,label='-1')
plt.scatter(Q0,pp2,label='2')
plt.scatter(Q0,pp_2,label='-2')

#plt.plot(pn[1],label='2')
#plt.plot(p_n[0],label='-1')
#plt.plot(p_n[1],label='-2')
#plt.plot(y)
plt.legend()

In [ ]:
spectra.load(Jfiles[9][0])
x=np.array(spectra.bias)
y=np.array(spectra.conductance)
y=y/y[0]
#plt.plot(1000*x,y)

R1=1000000
R2=1
C1=0.001
C2=0.3
Q0=0.7
Delta=0.4
delta_t=1.3
delta_s=0.0
T=0.1
eta=0.01

V=np.linspace(-7,7,80)
p0,pn,p_n=all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,2)

plt.plot(p0,label='0')
plt.plot(pn[0],label='1')
plt.plot(pn[1],label='2')
plt.plot(p_n[0],label='-1')
plt.plot(p_n[1],label='-2')

plt.legend()


In [ ]:
1e-21/1.6e-19

### Other stuff

In [ ]:
C1=[]
Q0=[]
ther=[]
C2=[]
h=0
for i in range(len(dat)):
    spectra.load(Tfiles[i][T[i].index(1.3)])
    x=np.array(spectra.bias)
    y=np.array(spectra.conductance)
    y=y/np.sum(y)
    plt.plot(1000*x,y+0.005*h)
    par=sc.optimize.curve_fit(lambda xx, C1,Q0,T,C2: current(xx, 1,10000, C1,C2,Q0,T), 1000*x, y,p0=[0.9,0.0,1,0.001],bounds=([0.1,-0.45,0.64,0.0005],[1.0,0.45,1.5,0.001]))[0]
    a=current(1000*x,1,10000,par[0],par[3],par[1],par[2])
    plt.plot(1000*x,a+0.005*h)
    h+=1
    C1.append(par[0])
    Q0.append(par[1])
    ther.append(par[2])
    C2.append(par[3])


In [ ]:
C1=[]
Q0=[]
C2=[]
h=0
for i in range(len(dat)):
    spectra.load(Tfiles[i][T[i].index(8.0)])
    x=np.array(spectra.bias)
    y=np.array(spectra.conductance)
    y=y/np.sum(y)
    plt.plot(1000*x,y+0.005*h)
    par=sc.optimize.curve_fit(lambda xx, C1,Q0,C2: current(xx, 1,10000, C1,C2,Q0,1.3), 1000*x, y,p0=[0.9,0.0,0.001],bounds=([0.1,-0.5,0.0005],[1.0,0.5,0.001]))[0]
    a=current(1000*x,1,10000,par[0],par[2],par[1],1.3)
    plt.plot(1000*x,a+0.005*h)
    h+=1
    C1.append(par[0])
    Q0.append(par[1])
    C2.append(par[2])

In [ ]:
spectra.load(Tfiles[0][T[0].index(1.3)])
x=np.array(spectra.bias)
y=np.array(spectra.conductance)
y=y/np.sum(y)
plt.plot(1000*x,y)

In [ ]:
ther=[]
Delta_island=[]
Q0=[]
h=0
for i in range(len(dat)):
    print(i)
    spectra.load(Tfiles[i][T[i].index(1.3)])
    x=np.array(spectra.bias)
    y=np.array(spectra.conductance)
    y=y/np.sum(y)
    plt.plot(1000*x,y+0.005*h)
    par=sc.optimize.curve_fit(lambda xx, Q0,T,Delta: currentD(xx, 1,10000, C1[i],C2[i],Q0,Delta,T), 1000*x, y,p0=[0.9,0.0,1.0],bounds=([0.1,-0.45,0.01],[1.0,0.45,1.3]))[0]
    a=currentD(1000*x,1,10000, C1[i],C2[i],par[0],par[2],par[1])
    plt.plot(1000*x,a+0.005*h)
    h+=1
    Q0.append(par[0])
    Delta_island.append(par[2])
    ther.append(par[1])


In [ ]:
y=[]
for i in Bfiles[1]:
    spectra.load(i)
    y.append(spectra.conductance/spectra.conductance[0])
x=np.array(spectra.bias)
plt.imshow(y,aspect='auto',origin='lower',extent=(1000*x[0],1000*x[-1],a[-1],a[0]))
plt.ylabel('B(mT)')
plt.xlabel('Bias (meV)')

## Low impedance

In [384]:
def fermi(T,x):
    return np.divide(1,1+np.exp(x/T))

def E1(C1,C2,V,n,Q0):
    k=C2/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def E2(C1,C2,V,n,Q0):
    k=C1/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def Gamma1(V,R1,C1,C2,n,Q0,T):
    k=C2/(C1+C2)
    Ec=(C1+C2)/(2*C1*C2)
    x=E1(C1,C2,V,n,Q0)
    return (x/(1-np.exp(-x/T)))/R1

def Gamma2(V,R2,C1,C2,n,Q0,T):
    k=C1/(C1+C2)
    Ec=(C1+C2)/(2*C1*C2)
    x=E2(C1,C2,V,-n,-Q0)
    return (x/(1-np.exp(-x/T)))/R2


def PN(V,R1,R2,C1,C2,Q0,T,n):
    mn=(Gamma1(V,R1,C1,C2,-n,Q0,T)+Gamma2(-V,R2,C1,C2,n,-Q0,T))/(Gamma1(-V,R1,C1,C2,n+1,-Q0,T)+Gamma2(V,R2,C1,C2,-n-1,Q0,T))
    en=(Gamma1(-V,R1,C1,C2,-n,-Q0,T)+Gamma2(V,R2,C1,C2,n,Q0,T))/(Gamma1(V,R1,C1,C2,n+1,Q0,T)+Gamma2(-V,R2,C1,C2,-n-1,-Q0,T))
    return en,mn

def check_p(V,R1,R2,C1,C2,Q0,T):
    Vmax=V
    n=[0]
    while True:
        a=[]
        b=[]
        for i in n:
            an,bn=PN(Vmax,R1,R2,C1,C2,Q0,T,i)
            a.append(an[0])
            b.append(bn[0])
        p0=1
        for i in range(len(a)):
            temp=np.zeros(len(a))
            temp[0:i+1]=1
            temp=temp.tolist()
            p0+=np.prod(a,where=temp)+np.prod(b,where=temp)
        p0=1/p0
        pn=p0*np.prod(a)
        p_n=p0*np.prod(b)
        #print(pn,p_n)
        n.append(n[-1]+1)
        if np.sum(pn)<0.001 and np.sum(p_n)<0.001:
            #print(len(a))
            break 
    return n[-1]

def all_p(V,R1,R2,C1,C2,Q0,T,n):
    a=[]
    b=[]
    for i in range(n):
        an,bn=PN(V,R1,R2,C1,C2,Q0,T,i)
        a.append(an)
        b.append(bn)
    p0=1
    pn=[]
    p_n=[]
    for i in range(n):
        temp=np.zeros(n)
        temp[0:i+1]=1
        temp=temp.tolist()
        temp1=np.full(len(V),1.0)
        temp2=np.full(len(V),1.0)
        for j in range(n):
            if temp[j]<=0.1:
                pass
            else:
                temp1*=a[j]       
                temp2*=b[j]
        p0+=temp1+temp2
        pn.append(temp1)
        p_n.append(temp2)
    p0=1/p0
    pn=p0*np.array(pn)
    p_n=p0*np.array(p_n)
    return p0,pn,p_n

def G1n(V,R1,C1,C2,Q0,T,n):
    return Gamma1(V,R1,C1,C2,n,Q0,T)-Gamma1(-V,R1,C1,C2,-n,-Q0,T)

def G2n(V,R2,C1,C2,Q0,T,n):
    return Gamma2(V,R2,C1,C2,n,Q0,T)-Gamma2(-V,R2,C1,C2,-n,-Q0,T)

def current(V,R1,R2,C1,C2,Q0,T):
    n=check_p(V,R1,R2,C1,C2,Q0,T)
    p0,pn,p_n=all_p(V,R1,R2,C1,C2,Q0,T,n)
    I=p0*G1n(V,R1,C1,C2,Q0,T,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G1n(V,R1,C1,C2,Q0,T,i)
        I+=p_n[i-1]*G1n(V,R1,C1,C2,Q0,T,-i)
    I=-np.gradient(I)
    return I/np.abs(I[0])


def current2(V,R1,R2,C1,C2,Q0,T):
    n=check_p(V,R1,R2,C1,C2,Q0,T)
    p0,pn,p_n=all_p(V,R1,R2,C1,C2,Q0,T,n)
    I=p0*G2n(V,R2,C1,C2,Q0,T,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G2n(V,R2,C1,C2,Q0,T,i)
        I+=p_n[i-1]*G2n(V,R2,C1,C2,Q0,T,-i)
    return I/np.abs(I[0])

## Low impedance Superconducting tip

In [ ]:
def bcs(delta,x,eta):
    if delta<=0.01: 
        return 1
    else:
        return (np.sign(x))*np.imag(np.divide(np.abs(x+eta*1j),np.sqrt(delta-(x+eta*1j)**2)))
        

def fermi(T,x):
    return np.divide(1,1+np.exp(x/T))

def E1(C1,C2,V,n,Q0):
    k=C2/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def E2(C1,C2,V,n,Q0):
    k=C1/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def Gamma1D(V,R1,C1,C2,n,Q0,Delta,delta_t,T,eta):
    x=np.linspace(-4*np.max(V),4*np.max(V),7000)
    a,b=np.meshgrid(x,E1(C1,C2,V,n,Q0))
    t=a+b
    return np.dot( (1-fermi(T,t)),bcs(Delta,x,eta)*fermi(T,x) )/R1

def Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,T,eta):
    x=np.linspace(-4*np.max(V),4*np.max(V),7000)
    a,b=np.meshgrid(x,E2(C1,C2,V,-n,-Q0))
    t=a+b
    return np.dot(1-fermi(T,t),fermi(T,x))/R2


def PND(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,n):
    mn=(Gamma1D(V,R1,C1,C2,-n,Q0,Delta,delta_t,T,eta)+Gamma2D(-V,R2,C1,C2,n,-Q0,Delta,delta_t,T,eta))/(Gamma1D(-V,R1,C1,C2,n+1,-Q0,Delta,delta_t,T,eta)+Gamma2D(V,R2,C1,C2,-n-1,Q0,Delta,delta_t,T,eta))
    en=(Gamma1D(-V,R1,C1,C2,-n,-Q0,Delta,delta_t,T,eta)+Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,T,eta))/(Gamma1D(V,R1,C1,C2,n+1,Q0,Delta,delta_t,T,eta)+Gamma2D(-V,R2,C1,C2,-n-1,-Q0,Delta,delta_t,T,eta))
    return en,mn

def check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta):
    Vmax=V[-1]
    n=[0]
    while True:
        a=[]
        b=[]
        for i in n:
            an,bn=PND(Vmax,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,i)
            a.append(an[0])
            b.append(bn[0])
        p0=1
        for i in range(len(a)):
            temp=np.zeros(len(a))
            temp[0:i+1]=1
            temp=temp.tolist()
            p0+=np.prod(a,where=temp)+np.prod(b,where=temp)
        p0=1/p0
        pn=p0*np.prod(a)
        p_n=p0*np.prod(b)
        #print(pn,p_n)
        n.append(n[-1]+1)
        if pn<0.0001 and p_n<0.0001:
            #print(len(a))
            break 
    return n[-1]

def all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,n):
    a=[]
    b=[]
    for i in range(n):
        an,bn=PND(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,i)
        a.append(an)
        b.append(bn)
    p0=1
    pn=[]
    p_n=[]
    for i in range(n):
        temp=np.zeros(n)
        temp[0:i+1]=1
        temp=temp.tolist()
        temp1=np.full(len(V),1.0)
        temp2=np.full(len(V),1.0)
        for j in range(n):
            if temp[j]<=0.1:
                pass
            else:
                temp1*=a[j]       
                temp2*=b[j]
        p0+=temp1+temp2
        pn.append(temp1)
        p_n.append(temp2)
    p0=1/p0
    pn=p0*np.array(pn)
    p_n=p0*np.array(p_n)
    return p0,pn,p_n

def G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,n):
    return Gamma1D(V,R1,C1,C2,n,Q0,Delta,delta_t,T,eta)-Gamma1D(-V,R1,C1,C2,-n,-Q0,Delta,delta_t,T,eta)

def G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,n):
    return Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,T,eta)-Gamma2D(-V,R2,C1,C2,-n,-Q0,Delta,delta_t,T,eta)

def currentD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta):
    n=check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta)
    p0,pn,p_n=all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,n)
    I=p0*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,i)
        I+=p_n[i-1]*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,-i)
    t=np.gradient(I)
    return t/np.sum(t)


def currentD2(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta):
    n=check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta)
    p0,pn,p_n=all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,n)
    I=p0*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,i)
        I+=p_n[i-1]*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,-i)
    t=np.gradient(I)
    return t/np.sum(t)


## High impedance

In [ ]:
def fermi(T,x):
    return np.divide(1,1+np.exp(x/T))

def E1(C1,C2,V,n,Q0):
    k=C2/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def E2(C1,C2,V,n,Q0):
    k=C1/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def Gamma1(V,R1,C1,C2,n,Q0,T):
    k=C2/(C1+C2)
    Ec=(C1+C2)/(2*C1*C2)
    x=E1(C1,C2,V,n,Q0)-k**2*Ec
    return (x/(1-np.exp(-x/T)))/R1

def Gamma2(V,R2,C1,C2,n,Q0,T):
    k=C1/(C1+C2)
    Ec=(C1+C2)/(2*C1*C2)
    x=E2(C1,C2,V,-n,-Q0)-k**2*Ec
    return (x/(1-np.exp(-x/T)))/R2


def PN(V,R1,R2,C1,C2,Q0,T,n):
    mn=(Gamma1(V,R1,C1,C2,-n,Q0,T)+Gamma2(-V,R2,C1,C2,n,-Q0,T))/(Gamma1(-V,R1,C1,C2,n+1,-Q0,T)+Gamma2(V,R2,C1,C2,-n-1,Q0,T))
    en=(Gamma1(-V,R1,C1,C2,-n,-Q0,T)+Gamma2(V,R2,C1,C2,n,Q0,T))/(Gamma1(V,R1,C1,C2,n+1,Q0,T)+Gamma2(-V,R2,C1,C2,-n-1,-Q0,T))
    return en,mn

def check_p(V,R1,R2,C1,C2,Q0,T):
    Vmax=V
    n=[0]
    while True:
        a=[]
        b=[]
        for i in n:
            an,bn=PN(Vmax,R1,R2,C1,C2,Q0,T,i)
            a.append(an[0])
            b.append(bn[0])
        p0=1
        for i in range(len(a)):
            temp=np.zeros(len(a))
            temp[0:i+1]=1
            temp=temp.tolist()
            p0+=np.prod(a,where=temp)+np.prod(b,where=temp)
        p0=1/p0
        pn=p0*np.prod(a)
        p_n=p0*np.prod(b)
        #print(pn,p_n)
        n.append(n[-1]+1)
        if np.sum(pn)<0.001 and np.sum(p_n)<0.001:
            #print(len(a))
            break 
    return n[-1]

def all_p(V,R1,R2,C1,C2,Q0,T,n):
    a=[]
    b=[]
    for i in range(n):
        an,bn=PN(V,R1,R2,C1,C2,Q0,T,i)
        a.append(an)
        b.append(bn)
    p0=1
    pn=[]
    p_n=[]
    for i in range(n):
        temp=np.zeros(n)
        temp[0:i+1]=1
        temp=temp.tolist()
        temp1=np.full(len(V),1.0)
        temp2=np.full(len(V),1.0)
        for j in range(n):
            if temp[j]<=0.1:
                pass
            else:
                temp1*=a[j]       
                temp2*=b[j]
        p0+=temp1+temp2
        pn.append(temp1)
        p_n.append(temp2)
    p0=1/p0
    pn=p0*np.array(pn)
    p_n=p0*np.array(p_n)
    return p0,pn,p_n

def G1n(V,R1,C1,C2,Q0,T,n):
    return Gamma1(V,R1,C1,C2,n,Q0,T)-Gamma1(-V,R1,C1,C2,-n,-Q0,T)

def G2n(V,R2,C1,C2,Q0,T,n):
    return Gamma2(V,R2,C1,C2,n,Q0,T)-Gamma2(-V,R2,C1,C2,-n,-Q0,T)

def current(V,R1,R2,C1,C2,Q0,T):
    n=check_p(V,R1,R2,C1,C2,Q0,T)
    p0,pn,p_n=all_p(V,R1,R2,C1,C2,Q0,T,n)
    I=p0*G1n(V,R1,C1,C2,Q0,T,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G1n(V,R1,C1,C2,Q0,T,i)
        I+=p_n[i-1]*G1n(V,R1,C1,C2,Q0,T,-i)
    t=np.gradient(I)
    return len(t)*t/np.sum(t)


def current2(V,R1,R2,C1,C2,Q0,T):
    n=check_p(V,R1,R2,C1,C2,Q0,T)
    p0,pn,p_n=all_p(V,R1,R2,C1,C2,Q0,T,n)
    I=p0*G2n(V,R2,C1,C2,Q0,T,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G2n(V,R2,C1,C2,Q0,T,i)
        I+=p_n[i-1]*G2n(V,R2,C1,C2,Q0,T,-i)
    t=np.gradient(I)
    return len(t)*t/np.sum(t)

## Experimental broadening

In [ ]:
def conv_lockin(V,R1,R2,C1,C2,Q0,T,Vw):
    x=np.linspace(-Vw,Vw,len(V))
    a,b=np.meshgrid(V,x)
    conv=[]
    for i in V:
        f=current(i+x,R1,R2,C1,C2,Q0,T)
        g=(Vw**2-x**2)**(3/2)/(Vw**2)
        conv.append(np.sum(f*g))
    np.array(conv)
    out=np.gradient(conv)
    return out/np.sum(out)

In [ ]:
x=np.linspace(-10.0,10.0,1000)
plt.plot(x,current(x,1,10000,0.3,0.01,0.0,0.1))
plt.plot(x,conv_lockin(x,1,10000,0.3,0.01,0.0,0.1,0.1))

## Low Impedance Superconductivity

In [ ]:
def bcs(delta,x,eta):
    if delta<=0.01: 
        return 1
    else:
        return (np.sign(x))*np.imag(np.divide(np.abs(x+eta*1j),np.sqrt(delta-(x+eta*1j)**2)))
        

def fermi(T,x):
    return np.divide(1,1+np.exp(x/T))

def E1(C1,C2,V,n,Q0):
    k=C2/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def E2(C1,C2,V,n,Q0):
    k=C1/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def Gamma1D(V,R1,C1,C2,n,Q0,Delta,delta_t,T,eta):
    x=np.linspace(-4*np.max(V),4*np.max(V),7000)
    a,b=np.meshgrid(x,E1(C1,C2,V,n,Q0))
    t=a+b
    return np.dot( bcs(delta_t,t,eta)*(1-fermi(T,t)),bcs(Delta,x,eta)*fermi(T,x) )/R1

def Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,T,eta):
    x=np.linspace(-4*np.max(V),4*np.max(V),7000)
    a,b=np.meshgrid(x,E2(C1,C2,V,-n,-Q0))
    t=a+b
    return np.dot(1-fermi(T,t),bcs(Delta,x,eta)*fermi(T,x))/R2


def PND(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,n):
    mn=(Gamma1D(V,R1,C1,C2,-n,Q0,Delta,delta_t,T,eta)+Gamma2D(-V,R2,C1,C2,n,-Q0,Delta,delta_t,T,eta))/(Gamma1D(-V,R1,C1,C2,n+1,-Q0,Delta,delta_t,T,eta)+Gamma2D(V,R2,C1,C2,-n-1,Q0,Delta,delta_t,T,eta))
    en=(Gamma1D(-V,R1,C1,C2,-n,-Q0,Delta,delta_t,T,eta)+Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,T,eta))/(Gamma1D(V,R1,C1,C2,n+1,Q0,Delta,delta_t,T,eta)+Gamma2D(-V,R2,C1,C2,-n-1,-Q0,Delta,delta_t,T,eta))
    return en,mn

def check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta):
    Vmax=V[-1]
    n=[0]
    while True:
        a=[]
        b=[]
        for i in n:
            an,bn=PND(Vmax,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,i)
            a.append(an[0])
            b.append(bn[0])
        p0=1
        for i in range(len(a)):
            temp=np.zeros(len(a))
            temp[0:i+1]=1
            temp=temp.tolist()
            p0+=np.prod(a,where=temp)+np.prod(b,where=temp)
        p0=1/p0
        pn=p0*np.prod(a)
        p_n=p0*np.prod(b)
        #print(pn,p_n)
        n.append(n[-1]+1)
        if pn<0.0001 and p_n<0.0001:
            #print(len(a))
            break 
    return n[-1]

def all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,n):
    a=[]
    b=[]
    for i in range(n):
        an,bn=PND(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,i)
        a.append(an)
        b.append(bn)
    p0=1
    pn=[]
    p_n=[]
    for i in range(n):
        temp=np.zeros(n)
        temp[0:i+1]=1
        temp=temp.tolist()
        temp1=np.full(len(V),1.0)
        temp2=np.full(len(V),1.0)
        for j in range(n):
            if temp[j]<=0.1:
                pass
            else:
                temp1*=a[j]       
                temp2*=b[j]
        p0+=temp1+temp2
        pn.append(temp1)
        p_n.append(temp2)
    p0=1/p0
    pn=p0*np.array(pn)
    p_n=p0*np.array(p_n)
    return p0,pn,p_n

def G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,n):
    return Gamma1D(V,R1,C1,C2,n,Q0,Delta,delta_t,T,eta)-Gamma1D(-V,R1,C1,C2,-n,-Q0,Delta,delta_t,T,eta)

def G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,n):
    return Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,T,eta)-Gamma2D(-V,R2,C1,C2,-n,-Q0,Delta,delta_t,T,eta)

def currentD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta):
    n=check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta)
    p0,pn,p_n=all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,n)
    I=p0*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,i)
        I+=p_n[i-1]*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,-i)
    t=np.gradient(I)
    return t/np.sum(t)


def currentD2(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta):
    n=check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta)
    p0,pn,p_n=all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,n)
    I=p0*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,i)
        I+=p_n[i-1]*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,-i)
    t=np.gradient(I)
    return t/np.sum(t)


In [ ]:
V=np.linspace(-10,10,100)
#R1=10000
#R2=1
#C1=0.01
#C2=0.5
#Delta=1.25
#Q0=0.1
#T=0.1
R1=10000
R2=1
C1=0.01
C2=0.5
Q0=0.1
Delta=0.2
T=0.1
spectra.load(Bfiles[4][5])
x=np.array(spectra.bias)
y=np.array(spectra.conductance)
y=y/y[0]
plt.plot(1000*x,y)
z=currentD(V,R1,R2,C1,C2,Q0,Delta,1.0,T,0.2)
z=z/z[0]
plt.plot(V,z)



In [ ]:
V=np.linspace(-2,2,100)
plt.plot(V,bcs(1,V))

In [ ]:

for i in range(9,len(dat)):
    print(i)
    spectra.load(Tfiles[i][T[i].index(1.3)])
    x=np.array(spectra.bias)
    y=np.array(spectra.conductance)
    y=y/np.sum(y)
    plt.plot(1000*x,y)
    par=sc.optimize.curve_fit(lambda xx, Q0,delta: currentD(xx,1000,1,C2[i],C1[i],Q0,delta,0.1), 1000*x, y,p0=[0.0,0.3],bounds=([-0.50,0.1],[0.5,1.3]))[0]
    a=currentD(1000*x,1000,1,C2[i],C1[i],par[0],par[1],0.1)
    plt.plot(1000*x,a)
    delta.append(par[1])
    Q0.append(par[0])

In [ ]:
i=7
spectra.load(Tfiles[i][T[i].index(1.3)])
x=np.array(spectra.bias)
y=np.array(spectra.conductance)
y=y/np.sum(y)
plt.plot(1000*x,y)
#a=currentD(1000*x,1000,1,C2[i],C1[i],Q0[i],delta[i],0.1)
#plt.plot(1000*x,a)

In [ ]:
temp1=[0.20538628915861032,0.3139708157071217,0.219155437068271,0.10000000000000002,0.9584898529396583,0.16640228862074904,0.3681922831136722,0.0,0.2097395898675521,0.458292984581787,0.10000000000000002,0.4714580736956538]
temp2=[373.0,578.0,419.0,363.0,865.0,846.0,873.0,305.0,613.0,1368.0,573.0,806.0]

plt.scatter(temp2,temp1)

## Low Impedance Superconductor Fixed

In [ ]:
def bcs(delta,x,eta):
    if delta<=0.01: 
        return 1
    else:
        return (np.sign(x))*np.imag(np.divide(np.abs(x+eta*1j),np.sqrt(delta-(x+eta*1j)**2)))
    
def bcs_i(delta,x,eta,n):
    if np.mod(n,2)==0:
        if delta<=0.01: 
            return 1
        else:
            return (np.sign(x))*np.imag(np.divide(np.abs(x+eta*1j),np.sqrt(delta-(x+eta*1j)**2)))
    else:
        return 1

def fermi(T,x):
    return np.divide(1,1+np.exp(x/T))

def E1(C1,C2,V,n,Q0):
    k=C2/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def E2(C1,C2,V,n,Q0):
    k=C1/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def Gamma1D(V,R1,C1,C2,n,Q0,Delta,delta_t,T,eta):
    x=np.linspace(-4*np.max(V),4*np.max(V),7000)
    a,b=np.meshgrid(x,E1(C1,C2,V,n,Q0))
    t=a+b
    return np.dot( bcs(delta_t,t,eta)*(1-fermi(T,t)),bcs_i(Delta,x,eta,n)*fermi(T,x) )/R1

def Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,T,eta):
    x=np.linspace(-4*np.max(V),4*np.max(V),7000)
    a,b=np.meshgrid(x,E2(C1,C2,V,-n,-Q0))
    t=a+b
    return np.dot(1-fermi(T,t),bcs_i(Delta,x,eta,n)*fermi(T,x))/R2


def PND(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,n):
    mn=(Gamma1D(V,R1,C1,C2,-n,Q0,Delta,delta_t,T,eta)+Gamma2D(-V,R2,C1,C2,n,-Q0,Delta,delta_t,T,eta))/(Gamma1D(-V,R1,C1,C2,n+1,-Q0,Delta,delta_t,T,eta)+Gamma2D(V,R2,C1,C2,-n-1,Q0,Delta,delta_t,T,eta))
    en=(Gamma1D(-V,R1,C1,C2,-n,-Q0,Delta,delta_t,T,eta)+Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,T,eta))/(Gamma1D(V,R1,C1,C2,n+1,Q0,Delta,delta_t,T,eta)+Gamma2D(-V,R2,C1,C2,-n-1,-Q0,Delta,delta_t,T,eta))
    return en,mn

def check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta):
    Vmax=V[-1]
    n=[0]
    while True:
        a=[]
        b=[]
        for i in n:
            an,bn=PND(Vmax,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,i)
            a.append(an[0])
            b.append(bn[0])
        p0=1
        for i in range(len(a)):
            temp=np.zeros(len(a))
            temp[0:i+1]=1
            temp=temp.tolist()
            p0+=np.prod(a,where=temp)+np.prod(b,where=temp)
        p0=1/p0
        pn=p0*np.prod(a)
        p_n=p0*np.prod(b)
        #print(pn,p_n)
        n.append(n[-1]+1)
        if pn<0.0001 and p_n<0.0001:
            #print(len(a))
            break 
    return n[-1]

def all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,n):
    a=[]
    b=[]
    for i in range(n):
        an,bn=PND(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,i)
        a.append(an)
        b.append(bn)
    p0=1
    pn=[]
    p_n=[]
    for i in range(n):
        temp=np.zeros(n)
        temp[0:i+1]=1
        temp=temp.tolist()
        temp1=np.full(len(V),1.0)
        temp2=np.full(len(V),1.0)
        for j in range(n):
            if temp[j]<=0.1:
                pass
            else:
                temp1*=a[j]       
                temp2*=b[j]
        p0+=temp1+temp2
        pn.append(temp1)
        p_n.append(temp2)
    p0=1/p0
    pn=p0*np.array(pn)
    p_n=p0*np.array(p_n)
    return p0,pn,p_n

def G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,n):
    return Gamma1D(V,R1,C1,C2,n,Q0,Delta,delta_t,T,eta)-Gamma1D(-V,R1,C1,C2,-n,-Q0,Delta,delta_t,T,eta)

def G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,n):
    return Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,T,eta)-Gamma2D(-V,R2,C1,C2,-n,-Q0,Delta,delta_t,T,eta)

def currentD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta):
    n=check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta)
    p0,pn,p_n=all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,n)
    I=p0*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,i)
        I+=p_n[i-1]*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,-i)
    t=np.gradient(I)
    return t/np.sum(t)


def currentD2(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta):
    n=check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta)
    p0,pn,p_n=all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,n)
    I=p0*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,i)
        I+=p_n[i-1]*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,T,eta,-i)
    t=np.gradient(I)
    return t/np.sum(t)


## Low Impedance Superconductivity Fixed 3 Delta

In [ ]:
def bcs(delta,x,eta):
    if delta<=0.01: 
        return 1.0
    else:
        return (np.sign(x))*np.imag(np.divide(np.abs(x+eta*1j),np.sqrt(delta**2-(x+eta*1j)**2)))
    
def bcs_i(delta,x,eta,n):
    if np.mod(n,2)==0:
        if delta<=0.01: 
            return 1.0
        else:
            return (np.sign(x))*np.imag(np.divide(np.abs(x+eta*1j),np.sqrt(delta**2-(x+eta*1j)**2)))
    else:
        return 1.0

def fermi(T,x):
    if T==0.0:
        return np.heaviside(-x,1)
    else:
        return np.divide(1,1+np.exp(x/T))

def E1(C1,C2,V,n,Q0):
    k=C2/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def E2(C1,C2,V,n,Q0):
    k=C1/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def Gamma1D(V,R1,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta):
    x=np.linspace(-4*np.max(V),4*np.max(V),7000)
    a,b=np.meshgrid(x,E1(C1,C2,V,n,Q0))
    t=a+b
    return np.dot( bcs(delta_t,t,eta)*(1-fermi(T,t)),bcs_i(Delta,x,eta,n)*fermi(T,x) )/R1

def Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta):
    x=np.linspace(-4*np.max(V),4*np.max(V),7000)
    a,b=np.meshgrid(x,E2(C1,C2,V,-n,-Q0))
    t=a+b
    return np.dot(bcs(delta_s,t,eta)*(1-fermi(T,t)),bcs_i(Delta,x,eta,n)*fermi(T,x))/R2


def PND(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    mn=(Gamma1D(V,R1,C1,C2,-n,Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(-V,R2,C1,C2,n,-Q0,Delta,delta_t,delta_s,T,eta))/(Gamma1D(-V,R1,C1,C2,n+1,-Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(V,R2,C1,C2,-n-1,Q0,Delta,delta_t,delta_s,T,eta))
    en=(Gamma1D(-V,R1,C1,C2,-n,-Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta))/(Gamma1D(V,R1,C1,C2,n+1,Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(-V,R2,C1,C2,-n-1,-Q0,Delta,delta_t,delta_s,T,eta))
    return en,mn

def check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta):
    Vmax=V[-1]
    n=[0]
    while True:
        a=[]
        b=[]
        for i in n:
            an,bn=PND(Vmax,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
            a.append(an[0])
            b.append(bn[0])
        p0=1
        for i in range(len(a)):
            temp=np.zeros(len(a))
            temp[0:i+1]=1
            temp=temp.tolist()
            p0+=np.prod(a,where=temp)+np.prod(b,where=temp)
        p0=1/p0
        pn=p0*np.prod(a)
        p_n=p0*np.prod(b)
        #print(pn,p_n)
        n.append(n[-1]+1)
        if pn<0.0001 and p_n<0.0001:
            #print(len(a))
            break 
    return n[-1]

def all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    a=[]
    b=[]
    for i in range(n):
        an,bn=PND(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
        a.append(an)
        b.append(bn)
    p0=1
    pn=[]
    p_n=[]
    for i in range(n):
        temp=np.zeros(n)
        temp[0:i+1]=1
        temp=temp.tolist()
        temp1=np.full(len(V),1.0)
        temp2=np.full(len(V),1.0)
        for j in range(n):
            if temp[j]<=0.1:
                pass
            else:
                temp1*=a[j]       
                temp2*=b[j]
        p0+=temp1+temp2
        pn.append(temp1)
        p_n.append(temp2)
    p0=1/p0
    pn=p0*np.array(pn)
    p_n=p0*np.array(p_n)
    return p0,pn,p_n

def G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    return Gamma1D(V,R1,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta)-Gamma1D(-V,R1,C1,C2,-n,-Q0,Delta,delta_t,delta_s,T,eta)

def G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    return Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta)-Gamma2D(-V,R2,C1,C2,-n,-Q0,Delta,delta_t,delta_s,T,eta)

def currentD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta):
    n=check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta)
    p0,pn,p_n=all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n)
    I=p0*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
        I+=p_n[i-1]*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,-i)
    t=np.gradient(I)
    return t/np.sum(t)


def currentD2(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta):
    n=check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta)
    p0,pn,p_n=all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n)
    I=p0*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
        I+=p_n[i-1]*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,-i)
    t=np.gradient(I)
    return t/np.sum(t)


In [ ]:
V=np.linspace(-15,15,200)
#R1=10000
#R2=1
#C1=0.01
#C2=0.5
#Delta=1.25
#Q0=0.1
#T=0.1
R1=10000
R2=1
C1=0.01
C2=0.3
Q0=0.8
Delta=0.60
T=0.1
y=y/y[0]
#plt.plot(1000*x,y)
z=currentD(V,R1,R2,C1,C2,Q0,Delta,1.0,0.2,T,0.1)
z=z/z[0]
#p0,pn,p_n=all_pD(V,R1,R2,C1,C2,Q0,Delta,1.0,0.2,T,0.1,5)
#plt.plot(V,p0,label='0')
#plt.plot(V,pn[0],label='1')
#plt.plot(V,pn[1],label='2')
#plt.plot(V,pn[2],label='3')
#plt.plot(V,p_n[0],label='-1')
#plt.plot(V,p_n[1],label='-2')
#plt.plot(V,p_n[3],label='-3')
plt.plot(V,z)
#plt.vlines(1/(2*C2)+Delta+1.0+0.2-0.5/(C1+C2),0,1)
#plt.vlines(1/(C2)+Q0,0,1)
plt.legend(loc=1)

# Adventure

# Josephson Pulse

In [ ]:
files=['S230803_738.dat','S230803_747.dat','S230803_753.dat','S230803_760.dat']
Pfiles=['C:/Users/jonor/Desktop/PhD/Lanak/Pb on Gr (analysis)/2023.07.26 - 1st evap/'+i for i in files]
Jfiles=['S230803_744.dat','S230803_752.dat','S230803_758.dat','S230803_764.dat']
PJfiles=['C:/Users/jonor/Desktop/PhD/Lanak/Pb on Gr (analysis)/2023.07.26 - 1st evap/'+i for i in Jfiles]


In [ ]:
ax1 = plt.subplot2grid((2,5), (0,0), colspan=2)
ax2 = plt.subplot2grid((2,5), (0,3), colspan=2)
ax3 = plt.subplot2grid((2,5), (1,0), colspan=2)
ax4 = plt.subplot2grid((2,5), (1,3), colspan=2)
ax=[ax1,ax2,ax3,ax4]
for i in range(len(PJfiles)):
    spectra.load(Pfiles[i])
    print(spectra.header.get('Lock-in>Amplitude'))
    x=np.array(spectra.bias)
    y=np.array(spectra.conductance)
    a1=min(find_nearest(x,-0.002),find_nearest(x,0.002))
    a2=max(find_nearest(x,-0.002),find_nearest(x,0.002))
    ax[i].plot(x[a1:a2]/x[a1],y[a1:a2]/y[a1])
    a1=min(find_nearest(x,-0.00015),find_nearest(x,0.00015))
    a2=max(find_nearest(x,-0.00015),find_nearest(x,0.00015))
    #ax[i].plot(x[a1:a2],y[a1:a2])
    #ax[i].axhline(0.0)
    #ax[i].vlines(0.8*150e-6,np.min(y[a1:a2]),np.max(y[a1:a2]))
    #ax[i].vlines(-0.8*150e-6,np.min(y[a1:a2]),np.max(y[a1:a2]))




In [ ]:
spectra.load(Pfiles[0])
x=np.array(spectra.bias)
y=np.array(spectra.conductance)
yy=np.array(spectra.current)
y=y/np.sum(np.abs(y))
plt.plot(1000*x,y)
print(np.mean(x[len(x)-20:-1]/yy[len(yy)-20:-1]))
R1=100000
R2=1
C1=0.001
C2=1.0
Q0=1.01
Delta=1.0
T=0.1
z=currentD(1000*x,R1,R2,C1,C2,Q0,Delta,1.25,0.0,T,0.1)
z=z/np.sum(np.abs(z))
plt.plot(1000*x,z)


In [ ]:
p0,pn,p_n=all_pD(1000*x,R1,R2,C1,C2,Q0,Delta,1.25,T,0.1,3)
a1=find_nearest(1000*x,4.58)
a2=find_nearest(1000*x,-4.58)
x=np.concatenate((x[0:a1],x[a2:-1]))
p0=np.concatenate((p0[0:a1],p0[a2:-1]))
p1=np.concatenate((pn[0][0:a1],pn[0][a2:-1]))
p_1=np.concatenate((p_n[0][0:a1],p_n[0][a2:-1]))
p2=np.concatenate((pn[1][0:a1],pn[1][a2:-1]))
p_2=np.concatenate((p_n[1][0:a1],p_n[1][a2:-1]))
plt.plot(1000*x,p0,label='0')
plt.plot(1000*x,p1,label='1')
plt.plot(1000*x,p_1,label='-1')
plt.plot(1000*x,p2,label='2')
plt.plot(1000*x,p_2,label='-2')
spectra.load(PJfiles[0])
x=np.array(spectra.bias)
y=np.array(spectra.conductance)
a1=min(find_nearest(x,-0.002),find_nearest(x,0.002))
a2=max(find_nearest(x,-0.002),find_nearest(x,0.002))
plt.plot(1000*x[a1:a2],y[a1:a2]/y[a1])


In [ ]:
f= lambda x, a: 1/(x**2+a**2)
g= lambda x, Vw: np.sqrt(Vw**2-x**2+0.00001*1j)*np.heaviside(Vw-np.abs(x),0)
#h= lambda x, a: (-fermi(1,x)+fermi(1,-x))*(a*np.heaviside(x,0)+np.heaviside(-x,0))
h= lambda x, a: (a*np.heaviside(x,0))-1

Vw=0.1
t=0.1
x=np.linspace(-10,10,5000)
V=np.linspace(-10,10,5000)
y=f(V,t)
a,b=np.meshgrid(x,V)
conv=np.dot(g(a-b,Vw),f(x,0.1))
deconv=np.dot(np.linalg.pinv(g(a-b,Vw)),f(V,0.1))
plt.plot(V,y/np.sum(np.abs(y)))
plt.plot(V,conv/np.sum(np.abs(conv)))
plt.plot(x,deconv/np.sum(np.abs(deconv)))
#plt.axhline(0.0)
#plt.vlines(0.0,-0.001,0.001)


# Proba

In [2]:
def bcs(delta,x,eta):
    if delta<=0.01: 
        return 1
    else:
        return (np.sign(x))*np.imag(np.divide(np.abs(x+eta*1j),np.sqrt(delta**2-(x+eta*1j)**2)))
    
def bcs_i(delta,x,eta,n):
    if np.mod(n,2)==0:
        if delta<=0.01: 
            return 1
        else:
            return (np.sign(x))*np.imag(np.divide(np.abs(x+eta*1j),np.sqrt(delta**2-(x+eta*1j)**2)))
    else:
        return 1
        

def fermi(T,x):
    if T==0.0:
        return np.heaviside(-x,1)
    else:
        return np.divide(1,1+np.exp(x/T))

def E1(C1,C2,V,n,Q0,Delta):
    k=C2/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)+(1-(-1)**n)*Delta/2

def E2(C1,C2,V,n,Q0,Delta):
    k=C1/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)+(1-(-1)**n)*Delta/2

def Gamma1D(V,R1,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta):
    x=np.linspace(-4*np.max(V),4*np.max(V),10000)
    a,b=np.meshgrid(x,E1(C1,C2,V,n,Q0,Delta))
    t=a+b
    return np.dot( bcs_i(Delta,t,eta,n)*(fermi(T,-t)),bcs(delta_t,x,eta)*fermi(T,x) )/R1

def Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta):
    x=np.linspace(-4*np.max(V),4*np.max(V),10000)
    a,b=np.meshgrid(x,E2(C1,C2,V,-n,-Q0,Delta))
    t=a+b
    return np.dot(bcs(delta_s,t,eta)*(fermi(T,-t)),bcs_i(Delta,x,eta,n)*fermi(T,x))/R2


def PND(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    mn=(Gamma1D(V,R1,C1,C2,-n,Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(-V,R2,C1,C2,n,-Q0,Delta,delta_t,delta_s,T,eta))/(Gamma1D(-V,R1,C1,C2,n+1,-Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(V,R2,C1,C2,-n-1,Q0,Delta,delta_t,delta_s,T,eta))
    en=(Gamma1D(-V,R1,C1,C2,-n,-Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta))/(Gamma1D(V,R1,C1,C2,n+1,Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(-V,R2,C1,C2,-n-1,-Q0,Delta,delta_t,delta_s,T,eta))
    return en,mn

def check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta):
    Vmax=V[-1]
    n=[0]
    while True:
        a=[]
        b=[]
        for i in n:
            an,bn=PND(Vmax,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
            a.append(an[0])
            b.append(bn[0])
        p0=1
        for i in range(len(a)):
            temp=np.zeros(len(a))
            temp[0:i+1]=1
            temp=temp.tolist()
            p0+=np.prod(a,where=temp)+np.prod(b,where=temp)
        p0=1/p0
        pn=p0*np.prod(a)
        p_n=p0*np.prod(b)
        #print(pn,p_n)
        n.append(n[-1]+1)
        if pn<0.0001 and p_n<0.0001:
            #print(len(a))
            break 
    return n[-1]

def all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    a=[]
    b=[]
    for i in range(n):
        an,bn=PND(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
        a.append(an)
        b.append(bn)
    p0=1
    pn=[]
    p_n=[]
    for i in range(n):
        temp1=np.full(len(V),1.0)
        temp2=np.full(len(V),1.0)
        for j in range(i+1):
            temp1*=a[j]     
            temp2*=b[j]
        p0+=temp1+temp2
        pn.append(temp1)
        p_n.append(temp2)
    p0=1/p0
    pn=p0*np.array(pn)
    p_n=p0*np.array(p_n)
    return p0,pn,p_n

def G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    return Gamma1D(V,R1,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta)-Gamma1D(-V,R1,C1,C2,-n,-Q0,Delta,delta_t,delta_s,T,eta)

def G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    return Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta)-Gamma2D(-V,R2,C1,C2,-n,-Q0,Delta,delta_t,delta_s,T,eta)

def currentD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta):
    n=check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta)
    p0,pn,p_n=all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n)
    I=p0*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
        I+=p_n[i-1]*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,-i)
    t=np.gradient(I)
    return t/np.sum(t)


def currentD2(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta):
    n=check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta)
    p0,pn,p_n=all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n)
    I=p0*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
        I+=p_n[i-1]*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,-i)
    t=np.gradient(I)
    return t/np.sum(t)


In [6]:
R1=10000
R2=1
C1=0.00001
C2=0.5
Q0=0.2
Delta=1


delta_t=1.3
delta_s=0.0
T=0.1
eta=0.05
N=100
V=np.linspace(-10,10,N)
pp0=[]
pp1=[]
pp_1=[]
pp2=[]
pp_2=[]
half=int(N/2)
for i in Q0:
    print(i)
    p0,pn,p_n=all_pD(V,R1,R2,C1,C2,i,Delta,delta_t,delta_s,T,eta,7)
    pp0.append(p0[half])
    pp1.append(pn[0][half])
    pp2.append(pn[1][half])
    pp_1.append(p_n[0][half])
    pp_2.append(p_n[1][half])

plt.scatter(Q0,pp0,label='0')
plt.scatter(Q0,pp1,label='1')
plt.scatter(Q0,pp_1,label='-1')
plt.scatter(Q0,pp2,label='2')
plt.scatter(Q0,pp_2,label='-2')

#plt.plot(pn[1],label='2')
#plt.plot(p_n[0],label='-1')
#plt.plot(p_n[1],label='-2')
#plt.plot(y)
#plt.legend()
#plt.vlines(-Delta+delta_t-1/(2*(C2)),0,3)
#plt.vlines(-Delta-delta_t-1/(2*(C2)),0,3)
#plt.vlines(-Delta-delta_t+1/(2*(C2)),0,0.06)
#plt.vlines(Delta+delta_t+3/(2*C2),0,0.06)

#spectra.load(Bfiles[1][0])
#x=np.array(spectra.bias)
#y=np.array(spectra.conductance)
#y=y/y[0]
#plt.plot(1000*x,y)
#y=currentD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta)
#plt.plot(V,y/y[0])
#z=np.gradient(G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,0))
#z1=np.gradient(G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,-1))
#plt.plot(V,z/z[0])
#plt.plot(V,z1/z1[0])

TypeError: 'float' object is not iterable

In [ ]:
def bcs(delta,x,eta):
    if delta<=0.01: 
        return 1
    else:
        return (np.sign(x))*np.imag(np.divide(np.abs(x+eta*1j),np.sqrt(delta**2-(x+eta*1j)**2)))
    
def bcs_i(delta,x,eta,n):
    if int(np.mod(np.abs(np.around(n)),2))==0:
        if delta<=0.01: 
            return 1
        else:
            return (np.sign(x))*np.imag(np.divide(np.abs(x+eta*1j),np.sqrt(delta**2-(x+eta*1j)**2)))
    else:
        return 1

def fermi(T,x):
    if T==0.0:
        return np.heaviside(-x,1)
    else:
        return np.divide(1,1+np.exp(x/T))

def E1(C1,C2,V,n,Q0,Delta):
    k=C2/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def E2(C1,C2,V,n,Q0,Delta):
    k=C1/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def Gamma1D(V,R1,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta):
    x=np.linspace(-4*np.max(V),4*np.max(V),50000)
    a,b=np.meshgrid(x,E1(C1,C2,V,n,Q0,Delta))
    t=a+b
    return np.dot( bcs_i(Delta,t,eta,n+Q0)*fermi(T,-t),fermi(T,x) )/R1

def Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta):
    x=np.linspace(-4*np.max(V),4*np.max(V),50000)
    a,b=np.meshgrid(x,E2(C1,C2,V,-n,-Q0,Delta))
    t=a+b
    return np.dot((fermi(T,-t)),bcs_i(Delta,x,eta,n+Q0)*fermi(T,x))/R2



def PND(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    mn=(Gamma1D(V,R1,C1,C2,-n,Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(-V,R2,C1,C2,n,-Q0,Delta,delta_t,delta_s,T,eta))/(Gamma1D(-V,R1,C1,C2,n+1,-Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(V,R2,C1,C2,-n-1,Q0,Delta,delta_t,delta_s,T,eta))
    en=(Gamma1D(-V,R1,C1,C2,-n,-Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta))/(Gamma1D(V,R1,C1,C2,n+1,Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(-V,R2,C1,C2,-n-1,-Q0,Delta,delta_t,delta_s,T,eta))
    return en,mn

def check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta):
    Vmax=V[-1]
    n=[0]
    while True:
        a=[]
        b=[]
        for i in n:
            an,bn=PND(Vmax,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
            a.append(an[0])
            b.append(bn[0])
        p0=1
        for i in range(len(a)):
            temp=np.zeros(len(a))
            temp[0:i+1]=1
            temp=temp.tolist()
            p0+=np.prod(a,where=temp)+np.prod(b,where=temp)
        p0=1/p0
        pn=p0*np.prod(a)
        p_n=p0*np.prod(b)
        #print(pn,p_n)
        n.append(n[-1]+1)
        if pn<0.0001 and p_n<0.0001:
            #print(len(a))
            break 
    return n[-1]

def all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    a=[]
    b=[]
    for i in range(n):
        an,bn=PND(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
        a.append(an)
        b.append(bn)
    p0=1
    pn=[]
    p_n=[]
    for i in range(n):
        temp1=np.full(len(V),1.0)
        temp2=np.full(len(V),1.0)
        for j in range(i+1):
            temp1*=a[j]     
            temp2*=b[j]
        p0+=temp1+temp2
        pn.append(temp1)
        p_n.append(temp2)
    p0=1/p0
    pn=p0*np.array(pn)
    p_n=p0*np.array(p_n)
    return p0,pn,p_n

def G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    return Gamma1D(V,R1,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta)-Gamma1D(-V,R1,C1,C2,-n,-Q0,Delta,delta_t,delta_s,T,eta)

def G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    return Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta)-Gamma2D(-V,R2,C1,C2,-n,-Q0,Delta,delta_t,delta_s,T,eta)

def currentD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta):
    n=check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta)
    p0,pn,p_n=all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n)
    I=p0*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
        I+=p_n[i-1]*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,-i)
    t=np.gradient(I)
    return t/np.sum(t)


def currentD2(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta):
    n=check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta)
    p0,pn,p_n=all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n)
    I=p0*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
        I+=p_n[i-1]*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,-i)
    t=np.gradient(I)
    return t/np.sum(t)


In [376]:
R1=10
R2=1
C1=0.01
C2=1.8
Q0=np.linspace(0.0,2.0,31)
Delta=1.2


delta_t=1.3
delta_s=0.0
T=0.1
eta=0.1
N=151
V=np.linspace(-5,5,N)
n=0
half=int(N/2)

#pp0=[]
#pp1=[]
#pp2=[]
#pp_1=[]
#pp_2=[]
#for i in Q0:
#    print(i)
#    p0,pn,p_n=all_pD(V,R1,R2,C1,C2,i,Delta,delta_t,delta_s,T,eta,4)
#    pp0.append(p0[half])
#    pp1.append(pn[0][half])
#    pp2.append(pn[1][half])
#    pp_1.append(p_n[0][half])
#    pp_2.append(p_n[1][half])
#
#plt.scatter(Q0,pp0,label='0')
#plt.scatter(Q0,pp1,label='1')
#plt.scatter(Q0,pp_1,label='-1')
#plt.scatter(Q0,pp2,label='2')
#plt.scatter(Q0,pp_2,label='-2')
#plt.legend()
spectra.load("C:/Users/jonor/Desktop/PhD/Lanak/Pb on Gr (analysis)/2023.07.26 - 1st evap/23.08.04 - New island1/23.08.04 - grids triangular/G02230804_247.dat")
plt.plot(1000*spectra.bias,spectra.conductance/spectra.conductance[0])
I=currentD(V,R1,R2,C1,C2,0.96,Delta,delta_t,delta_s,T,eta)
plt.plot(V,I/I[0])
#I=currentD2(V,R1,R2,C1,C2,0.0,Delta,delta_t,delta_s,T,eta)
#plt.plot(V,I)



In [ ]:
I0=G1nD(V,R1,R2,C1,C2,0.9,Delta,delta_t,delta_s,T,eta,0)
I1=G1nD(V,R1,R2,C1,C2,0.9,Delta,delta_t,delta_s,T,eta,-1)
I2=G1nD(V,R1,R2,C1,C2,0.9,Delta,delta_t,delta_s,T,eta,-2)
plt.vlines(delta_t+Delta-1/(2*C2),0,0.001)
plt.vlines(-delta_t-Delta-3/(2*C2),0,0.001)
plt.plot(V,np.gradient(I0+I1))

In [ ]:
def bcs(delta,x,eta):
    if delta<=0.01: 
        return 1
    else:
        return (np.sign(x))*np.imag(np.divide(np.abs(x+eta*1j),np.sqrt(delta**2-(x+eta*1j)**2)))
    
#def bcs_i(delta,x,eta,n):
#    if int(np.mod(n,2))==0:
#        if delta<=0.01: 
#            return 1
#        else:
#            return (np.sign(x))*np.imag(np.divide(np.abs(x+eta*1j),np.sqrt(delta**2-(x+eta*1j)**2)))
#    else:
#        return 1

def bcs_i(delta,x,eta,n):
    if delta<=0.01: 
        return 1
    else:
        return (np.sign(x))*np.imag(np.divide(np.abs(x+eta*1j),np.sqrt(delta**2-(x+eta*1j)**2)))

def fermi(T,x):
    if T==0.0:
        return np.heaviside(-x,1)
    else:
        return np.divide(1,1+np.exp(x/T))

def E0(C1,C2,V,n,Q0,Delta):
    return (n+Q0)**2/(2*(C1+C2))+(1-(-1)**(n))*Delta/2+(C1*C2*V**2)/(C1+C2)

def E1(C1,C2,V,n,Q0,Delta):
    k=C2/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def E2(C1,C2,V,n,Q0,Delta):
    k=C1/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def Gamma1D(V,R1,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta):
    x=np.linspace(-4*np.max(V),4*np.max(V),10000)
    a,b=np.meshgrid(x,E1(C1,C2,V,n,Q0,Delta))
    t=a+b
    return np.dot( bcs_i(Delta,t,eta,n)*fermi(T,-t), bcs(delta_t,x,eta)*fermi(T,x) )/R1

def Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta):
    x=np.linspace(-4*np.max(V),4*np.max(V),10000)
    a,b=np.meshgrid(x,E2(C1,C2,V,-n,-Q0,Delta))
    t=a+b
    return np.dot((fermi(T,-t)),bcs_i(Delta,x,eta,n)*fermi(T,x))/R2



def PND(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    mn=(Gamma1D(V,R1,C1,C2,-n,Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(-V,R2,C1,C2,n,-Q0,Delta,delta_t,delta_s,T,eta))/(Gamma1D(-V,R1,C1,C2,n+1,-Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(V,R2,C1,C2,-n-1,Q0,Delta,delta_t,delta_s,T,eta))
    en=(Gamma1D(-V,R1,C1,C2,-n,-Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta))/(Gamma1D(V,R1,C1,C2,n+1,Q0,Delta,delta_t,delta_s,T,eta)+Gamma2D(-V,R2,C1,C2,-n-1,-Q0,Delta,delta_t,delta_s,T,eta))
    return en,mn

def check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta):
    V=V[-1]
    n=[0]
    while True:
        Z=0
        for i in range(-n[-1],n[-1]+1):
            Z+=np.exp(-E0(C1,C2,V,i,Q0,Delta)/T)
        pn=np.exp(-E0(C1,C2,V,n[-1],Q0,Delta)/T)/Z
        p_n=np.exp(-E0(C1,C2,V,-n[-1],Q0,Delta)/T)/Z
        n.append(n[-1]+1)
        if pn<0.0001 and p_n<0.0001:
            #print(len(a))
            break 
    return n[-1]

def all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    Z=0
    for i in range(-n,n+1):
        Z+=np.exp(-E0(C1,C2,V,i,Q0,Delta)/T)
    p0=np.exp(-E0(C1,C2,V,0,Q0,Delta)/T)/Z
    pn=[]
    p_n=[]
    for i in range(1,n+1):
        pn.append(np.exp(-E0(C1,C2,V,i,Q0,Delta)/T)/Z)
        p_n.append(np.exp(-E0(C1,C2,V,-i,Q0,Delta)/T)/Z)
    return p0,pn,p_n

def G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    return Gamma1D(V,R1,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta)-Gamma1D(-V,R1,C1,C2,-n,-Q0,Delta,delta_t,delta_s,T,eta)

def G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n):
    return Gamma2D(V,R2,C1,C2,n,Q0,Delta,delta_t,delta_s,T,eta)-Gamma2D(-V,R2,C1,C2,-n,-Q0,Delta,delta_t,delta_s,T,eta)

def currentD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta):
    n=check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta)
    p0,pn,p_n=all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n)
    I=p0*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
        I+=p_n[i-1]*G1nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,-i)
    t=np.gradient(I)
    return t


def currentD2(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta):
    n=check_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta)
    p0,pn,p_n=all_pD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,n)
    I=p0*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,i)
        I+=p_n[i-1]*G2nD(V,R1,R2,C1,C2,Q0,Delta,delta_t,delta_s,T,eta,-i)
    t=np.gradient(I)
    return t
